In [2]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [3]:
import sys
import os
sys.path.insert(0, "/var/home/d.svirchkov/custom_scripts/scripts/")

In [4]:
hdfs_path = "hdfs dfs" #"/usr/share/hadoop/bin/hdfs dfs"

In [5]:
full_path = '...'

---

In [6]:
selected_features = ['dc_inflow_ecom1_365days_op_type1_mean_amnt',
 'dc_inflow_ecom1_365days_mcc_cat7_mean_hourdiff',
 'dc_inflow_ecom1_365days_mcc_cat2_mean_amnt',
 'ac_inflow_ecom1_365days_op_type5_mean_hourdiff',
 'dc_inflow_ecom1_365days_mcc_cat4_mean_hourdiff',
 'dc_inflow_ecom1_365days_op_type1_mean_hourdiff',
 'dc_inflow_ecom1_365days_mcc_cat3_mean_amnt',
 'dc_inflow_ecom1_365days_mcc_cat4_cnt',
 'ac_inflow_ecom1_365days_mcc_cat7_total_amnt',
 'dc_inflow_ecom1_365days_op_type2_mean_amnt',
 'dc_inflow_ecom1_365days_op_type3_total_amnt',
 'dc_inflow_ecom1_365days_mcc_cat2_first_act',
 'dc_inflow_ecom1_365days_mcc_cat5_mean_hourdiff',
 'cc_outflow_ecom1_365days_op_type4_mean_amnt',
 'dc_inflow_ecom1_365days_mcc_cat6_mean_hourdiff',
 'dc_inflow_ecom1_365days_mcc_cat5_mean_amnt',
 'dc_inflow_ecom1_365days_mcc_cat9_mean_hourdiff',
 'dc_inflow_ecom1_365days_mcc_cat13_mean_hourdiff',
 'dc_inflow_ecom1_365days_op_type5_mean_amnt',
 'dc_inflow_ecom1_365days_op_type5_cnt',
 'dc_inflow_ecom1_365days_mcc_cat4_mean_amnt',
 'ac_inflow_ecom1_365days_mcc_cat2_total_amnt',
 'ac_inflow_ecom1_365days_op_type7_mean_hourdiff',
 'ac_inflow_ecom1_365days_mcc_cat4_total_amnt',
 'dc_inflow_ecom1_365days_mcc_cat1_mean_amnt',
 'dc_inflow_ecom1_365days_mcc_cat1_last_act',
 'dc_inflow_ecom2_365days_op_type2_mean_hourdiff',
 'cc_outflow_ecom1_365days_op_type4_first_act',
 'dc_inflow_ecom1_365days_op_type2_mean_hourdiff',
 'dc_inflow_ecom1_365days_mcc_cat7_mean_amnt'
]


In [7]:
best_params = {'boosting_type': 'gbdt',
 'objective': 'binary',
 'metric': 'auc',
 'num_leaves': 2**5,
 'max_depth': 5,
 'learning_rate': 0.05,
 'feature_fraction': 0.6,
 'bagging_fraction': 0.7,
 'bagging_freq': 1,
 'min_data_in_leaf': 30,
 'num_threads': 10,
 'n_estimators': 10000,
 'random_state': 32,
 'metric_freq': 10}

In [8]:
params = best_params.copy()

In [9]:
params['num_threads'] = 10

---

In [ ]:
train_target = pd.read_csv('./data/alfabattle2_train_target.csv')
train_target = train_target.rename(columns={'app_id': 'ID', 'flag': 'TARGET'})

In [ ]:
use_keys = ['mcc_cat' + str(i) for i in range(1, cntd_mcc)] + \
           ['op_type' + str(i) for i in range(1, cntd_op_type)] 

In [ ]:
X = train_target[['ID', 'product', 'TARGET']].copy()

for key in use_keys:
    
    print()
    print(f'key {key}')
    t0 = time.time()
    
    try:
        rdd = spark.read.parquet(full_path + 'stats_' + key)  
        usecols = ['ID'] + [i for i in rdd.columns if i in selected_features]
        XX = rdd.select(usecols).toPandas()
    except Exception as e:
        print(f'catch some Exception //', key)
        continue
    
    X.ID = X.ID.astype('int64')
    XX.ID = XX.ID.astype('int64')
    
    X = pd.merge(X, XX, on='ID', how='outer')  
    
    print('XX.shape', XX.shape, f'time: {time.time() - t0:.1f}')  

In [224]:
%%time
train_mask = X.TARGET.isin({0,1})
res = base_cv(
    X[train_mask], X[train_mask].TARGET, selected_features, 
    model_params=params, nfolds=5, base_train_seed=42, num_train_seeds=3, fold_seed=15,
    save_model_name='tmp_alfabank_v2', model_folder='../../models/', use_saved_model=False, 
)

(963811, 30)
1 it: 266 score: 0.74054 tot: 192763 target: 5316  time 11.3s
2 it: 179 score: 0.73757 tot: 192763 target: 5316  time 8.6s
3 it: 149 score: 0.74336 tot: 192762 target: 5315  time 8.2s
4 it: 207 score: 0.73954 tot: 192762 target: 5315  time 9.8s
5 it: 312 score: 0.73541 tot: 192761 target: 5315  time 12.4s
seed: 42 metric_avg: 0.73928 metric_full: 0.73923 folds_std: 0.00269  time: 50.8s
1 it: 158 score: 0.74003 tot: 192763 target: 5316  time 8.1s
2 it: 159 score: 0.73699 tot: 192763 target: 5316  time 7.8s
3 it: 241 score: 0.74469 tot: 192762 target: 5315  time 10.4s
4 it: 292 score: 0.74049 tot: 192762 target: 5315  time 12.0s
5 it: 183 score: 0.7376 tot: 192761 target: 5315  time 8.7s
seed: 52 metric_avg: 0.73996 metric_full: 0.73996 folds_std: 0.00272  time: 47.5s
1 it: 296 score: 0.74098 tot: 192763 target: 5316  time 11.7s
2 it: 279 score: 0.73869 tot: 192763 target: 5316  time 11.5s
3 it: 247 score: 0.74379 tot: 192762 target: 5315  time 10.5s
4 it: 151 score: 0.73931

---

`predict test`

In [160]:
test_target = pd.read_csv('./data/alfabattle2_test_target_contest.csv')
test_target['flag'] = -1
test_target = test_target.rename(columns={'app_id': 'ID', 'flag': 'TARGET'})

In [161]:
cntd_mcc = 28
cntd_op_type = 22

use_keys = ['mcc_cat' + str(i) for i in range(1, cntd_mcc)] + \
           ['op_type' + str(i) for i in range(1, cntd_op_type)] 

In [162]:
len(use_keys)

48

In [165]:
%%time
X = test_target[['ID', 'product', 'TARGET']].copy()

for key in use_keys:
    
    print()
    print(f'key {key}')
    t0 = time.time()
    
    try:
        rdd = spark.read.parquet(full_path + 'stats_' + key)  
        usecols = ['ID'] + [i for i in rdd.columns if i in selected_features]
        XX = rdd.select(usecols).toPandas()
    except Exception as e:
        print(f'catch exception at key')
        continue
    
    X.ID = X.ID.astype('int64')
    XX.ID = XX.ID.astype('int64')
    
    X = pd.merge(X, XX, on='ID', how='left')  # change to left  
    
    print('XX.shape', XX.shape, f'time: {time.time() - t0:.1f}')


key mcc_cat1
XX.shape (456067, 3) time: 12.3

key mcc_cat2
XX.shape (465678, 4) time: 4.6

key mcc_cat3
XX.shape (420127, 2) time: 3.8

key mcc_cat4
XX.shape (400838, 5) time: 4.1

key mcc_cat5
XX.shape (374651, 3) time: 3.7

key mcc_cat6
XX.shape (390240, 2) time: 3.5

key mcc_cat7
XX.shape (409954, 4) time: 4.2

key mcc_cat8
catch exception at key

key mcc_cat9
XX.shape (370570, 2) time: 3.3

key mcc_cat10
catch exception at key

key mcc_cat11
catch exception at key

key mcc_cat12
catch exception at key

key mcc_cat13
XX.shape (314246, 2) time: 3.1

key mcc_cat14
catch exception at key

key mcc_cat15
catch exception at key

key mcc_cat16
catch exception at key

key mcc_cat17
catch exception at key

key mcc_cat18
catch exception at key

key mcc_cat19
catch exception at key

key mcc_cat20
catch exception at key

key mcc_cat21
catch exception at key

key mcc_cat22
catch exception at key

key mcc_cat23
catch exception at key

key mcc_cat24
catch exception at key

key mcc_cat25
catch exc

In [217]:
X.to_csv('./data/X1_test.csv', index=0)

In [168]:
X.head()

,ID,product,TARGET,dc_inflow_ecom1_365days_mcc_cat1_mean_amnt,dc_inflow_ecom1_365days_mcc_cat1_last_act,dc_inflow_ecom1_365days_mcc_cat2_first_act,dc_inflow_ecom1_365days_mcc_cat2_mean_amnt,ac_inflow_ecom1_365days_mcc_cat2_total_amnt,dc_inflow_ecom1_365days_mcc_cat3_mean_amnt,dc_inflow_ecom1_365days_mcc_cat4_cnt,dc_inflow_ecom1_365days_mcc_cat4_mean_hourdiff,dc_inflow_ecom1_365days_mcc_cat4_mean_amnt,ac_inflow_ecom1_365days_mcc_cat4_total_amnt,dc_inflow_ecom1_365days_mcc_cat5_mean_amnt,dc_inflow_ecom1_365days_mcc_cat5_mean_hourdiff,dc_inflow_ecom1_365days_mcc_cat6_mean_hourdiff,dc_inflow_ecom1_365days_mcc_cat7_mean_amnt,dc_inflow_ecom1_365days_mcc_cat7_mean_hourdiff,ac_inflow_ecom1_365days_mcc_cat7_total_amnt,dc_inflow_ecom1_365days_mcc_cat9_mean_hourdiff,dc_inflow_ecom1_365days_mcc_cat13_mean_hourdiff,dc_inflow_ecom1_365days_op_type1_mean_hourdiff,dc_inflow_ecom1_365days_op_type1_mean_amnt,dc_inflow_ecom1_365days_op_type2_mean_hourdiff,dc_inflow_ecom1_365days_op_type2_mean_amnt,dc_inflow_ecom2_365days_op_type2_mean_hourdiff,dc_inflow_ecom1_365days_op_type3_total_amnt,cc_outflow_ecom1_365days_op_type4_first_act,cc_outflow_ecom1_365days_op_type4_mean_amnt,dc_inflow_ecom1_365days_op_type5_mean_amnt,dc_inflow_ecom1_365days_op_type5_cnt,ac_inflow_ecom1_365days_op_type5_mean_hourdiff,ac_inflow_ecom1_365days_op_type7_mean_hourdiff
0,1063620,0,-1,0.374893,5.0,339.0,0.463294,11.119046,0.383692,72.0,9.625000,0.339636,24.453808,0.374568,6.083333,10.526316,0.356436,6.900000,7.128711,12.279070,4.777778,NaN,NaN,9.271375,0.378780,15.671429,1.524605,355.0,0.529622,0.432579,26.0,9.192308,NaN
1,1063621,0,-1,0.333128,36.0,323.0,0.476891,9.537814,0.349947,1.0,1.000000,0.345966,0.345966,0.546607,65.000000,21.000000,0.358662,47.500000,2.151972,1.666667,NaN,NaN,NaN,71.269231,0.350639,NaN,0.927244,245.0,0.534908,0.395807,25.0,46.200000,NaN
2,1063622,1,-1,0.278751,1.0,359.0,0.402861,27.394582,0.279859,25.0,3.400000,0.290066,7.251658,0.329207,23.085714,4.251429,0.267015,8.695652,6.141336,14.800000,NaN,8.507317,0.277019,NaN,NaN,NaN,19.754887,321.0,0.432811,NaN,NaN,NaN,13.75
3,1063623,1,-1,0.370395,1.0,357.0,0.476424,64.793633,0.347554,39.0,11.512821,0.348874,13.606069,0.381429,10.000000,3.176471,0.359761,19.500000,11.512355,10.705882,NaN,12.117048,0.368174,10.000000,0.447771,NaN,58.572658,343.0,0.496986,0.411189,28.0,17.857143,15.40
4,1063624,2,-1,0.283374,42.0,359.0,0.470212,17.397832,0.151959,17.0,15.529412,0.312128,5.306176,0.375170,0.000000,NaN,0.299778,12.125000,2.398222,9.000000,NaN,NaN,NaN,30.262295,0.299435,44.000000,12.489300,355.0,0.288790,NaN,NaN,NaN,NaN


In [170]:
def predict_new_data(
    X_to_pred, num_folds=5, num_seeds=3, start_seed=42, model_name='', 
    model_folder='../../models/', seed_step=10
    ):
    
    features = pd.read_csv(model_folder + model_name + '/train_features.txt').columns.tolist()
    y_predicted = np.zeros(shape=(X_to_pred.shape[0], ))
    for fold in range(num_folds):
        for num_seed in range(num_seeds):
            seed = start_seed + num_seed * seed_step
            fname = model_folder + model_name + '/of_fold' + str(fold + 1) + '_seed' + str(seed) + '.model'
            model_loaded = lightgbm.Booster(model_file=fname)
            y_predicted += model_loaded.predict(X_to_pred[features].round(6)) / (num_folds * num_seeds)
    return np.round(y_predicted, decimals = 5)

In [172]:
model_name='tmp_alfabank_v2'
model_folder='../../models/'

In [183]:
preds = predict_new_data(X, model_name=model_name, model_folder=model_folder)

In [187]:
test_target.TARGET = preds

In [188]:
test_target.head()

,ID,product,TARGET
0,1063620,0,0.01241
1,1063621,0,0.02849
2,1063622,1,0.04758
3,1063623,1,0.02063
4,1063624,2,0.02509


In [198]:
# sub = test_target[['ID', 'TARGET']].rename(columns={'ID': 'app_id', 'TARGET': 'flag'})
# sub.to_csv('./submissions/submission.csv', index=False)

In [213]:
import random
random.seed(42)
a = 79410101000
b = 79999089999

result = test_target[['ID', 'TARGET']]
len_df = result.shape[0]
result.insert(1, 'fake_phone', [random.randint(a, b) for i in range(len_df)])

In [214]:
result.head()

,ID,fake_phone,TARGET
0,1063620,79529641831,0.01241
1,1063621,79436956092,0.02849
2,1063622,79705411485,0.04758
3,1063623,79673051628,0.02063
4,1063624,79649771711,0.02509


In [215]:
result.shape

(502716, 3)

In [216]:
result.to_csv('result.csv', index=0)